### Imports

In [1]:
Tf_Features = 10
One_Hot =  False
Standard_Scale = True
N_Gram = 1

In [2]:
def text_cleaner(text,all_stop):
    """
    clean_str = text_cleaner(dirty_string)
    """
    regex = re.compile('[%s]' % re.escape(string.punctuation))

    text = text.lower()
    text = text.replace('<!--DONOTREMOVEESSAYDIVIDER-->',' ')
    text = text.replace('\n' , ' ')
    text = regex.sub('',text)
    text = ' '.join([word for word in text.split() if word not in all_stop])
    return(text)

In [3]:
import gc
import re
import string
import calendar
import numpy as np
import pandas as pd
from nltk.corpus import stopwords

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.preprocessing import LabelEncoder

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import TfidfVectorizer

In [4]:
donor_type = {'Unnamed: 0':int, 'Donor ID': str, 
    'Donor City': str, 'Donor State': str, 'Donor Is Teacher': str, 'Donor Zip': str, 'Donations Count': int,
    'Donation Mean Amount': float, 'Donation Sum Amount': float, 'Donation Median Amount': float,
    'Donation Max Amount': float, 'Donation Min Amount': float, 'Donation Std Amount': float,
    'One Time Donor': int}

In [5]:
Donors = pd.read_csv('./Input/New/Donors_Clean.csv',dtype=donor_type)
del Donors['Unnamed: 0']
del donor_type

In [7]:
df = pd.read_csv('./Input/Processed/df_no_encode_yes_essay.csv')
del df['Unnamed: 0']

### Prelim EDA

In [8]:
for i in ['Project Type', 'Project Posted Year', 'Project Posted Month']:
    print('*****************')
    print(df.groupby(i)['Project Current Status'].mean())
    print(' ')
    print(df[i].value_counts())
    print(' ')

*****************
Project Type
Professional Development    0.865304
Student-Led                 0.886006
Teacher-Led                 0.772549
Name: Project Current Status, dtype: float64
 
Teacher-Led                 1009555
Student-Led                    6667
Professional Development       3816
Name: Project Type, dtype: int64
 
*****************
Project Posted Year
2013    0.732929
2014    0.749114
2015    0.775672
2016    0.726125
2017    0.782376
2018    0.996245
Name: Project Current Status, dtype: float64
 
2017    250037
2016    236517
2015    173545
2014    155963
2013    124877
2018     79099
Name: Project Posted Year, dtype: int64
 
*****************
Project Posted Month
Apr    0.772330
Aug    0.758185
Dec    0.840962
Feb    0.834153
Jan    0.837275
Jul    0.720440
Jun    0.693997
Mar    0.790286
May    0.704221
Nov    0.762642
Oct    0.755387
Sep    0.769610
Name: Project Current Status, dtype: float64
 
Sep    148904
Aug    133176
Oct    101904
Mar     96784
Jan     88886
N

In [9]:
print('*****************')
print(df.groupby('Project Grade Level Category')['Project Current Status'].mean())
print(' ')
print(df['Project Grade Level Category'].value_counts())

*****************
Project Grade Level Category
Grades 3-5       0.770101
Grades 6-8       0.766145
Grades 9-12      0.774374
Grades PreK-2    0.779500
unknown          0.695652
Name: Project Current Status, dtype: float64
 
Grades PreK-2    399950
Grades 3-5       336070
Grades 6-8       165453
Grades 9-12      118519
unknown              46
Name: Project Grade Level Category, dtype: int64


In [10]:
print('*****************')
print(df.groupby('Teacher Prefix')['Project Current Status'].mean())
print("  ")
print(df['Teacher Prefix'].value_counts())

*****************
Teacher Prefix
Dr.        0.832061
Mr.        0.777729
Mrs.       0.759930
Ms.        0.793272
Teacher    0.730658
Name: Project Current Status, dtype: float64
  
Mrs.       524423
Ms.        370966
Mr.        111229
Teacher     13158
Dr.           262
Name: Teacher Prefix, dtype: int64


In [11]:
print('*****************')
print(df.groupby('School Metro Type')['Project Current Status'].mean())
print(' ')
print(df['School Metro Type'].value_counts())

*****************
School Metro Type
rural       0.738133
suburban    0.755320
town        0.731276
unknown     0.760090
urban       0.799129
Name: Project Current Status, dtype: float64
 
urban       482205
suburban    311014
rural        99875
unknown      80826
town         46118
Name: School Metro Type, dtype: int64


In [47]:
df.groupby('School State')['Project Current Status'].mean()

School State
Alabama                 0.722669
Alaska                  0.798742
Arizona                 0.723093
Arkansas                0.729794
California              0.771017
Colorado                0.805016
Connecticut             0.824689
Delaware                0.830572
District of Columbia    0.868137
Florida                 0.747709
Georgia                 0.709277
Hawaii                  0.833333
Idaho                   0.791928
Illinois                0.826192
Indiana                 0.788410
Iowa                    0.723054
Kansas                  0.760597
Kentucky                0.738394
Louisiana               0.719402
Maine                   0.806434
Maryland                0.793831
Massachusetts           0.855406
Michigan                0.782064
Minnesota               0.817131
Mississippi             0.695693
Missouri                0.760883
Montana                 0.825565
Nebraska                0.726842
Nevada                  0.692561
New Hampshire           0.8152

### Encode / Scale

In [12]:
encode_cols = ['Project Type', 'Project Posted Year', 
            'Project Posted Month', 'Project Grade Level Category',
            'Project Resource Category', 'Teacher Prefix',
            'School Metro Type','School State',
            'School District', 'School Zip']

num_cols = ['Project Cost', 'School Percentage Free Lunch',
            'Project num Unique Resources','Total Resource Quantity',
            'Mean Resource Cost', 'Total Project Cost', 'Median Resource Cost',
            'Most exp Resource Cost', 'Least exp Resource Cost']

In [20]:
filt_df = df[['Project Cost']]

In [28]:
low = .01
high = .99
quant_df = filt_df.quantile([low, high])
print(quant_df)

      Project Cost
0.01      188.2828
0.99     1878.5200


In [29]:
filt_df = filt_df.apply(lambda x: x[(x>quant_df.loc[low,x.name]) & 
                                    (x < quant_df.loc[high,x.name])], axis=0)

In [30]:
df[num_cols].head()

,Project Cost,School Percentage Free Lunch,Project num Unique Resources,Total Resource Quantity,Mean Resource Cost,Total Project Cost,Median Resource Cost,Most exp Resource Cost,Least exp Resource Cost
0,361.80,95.0,4,28.0,59.992500,239.97,16.98,199.99,6.02
1,416.24,95.0,4,6.0,70.447500,281.79,40.40,199.99,1.00
2,829.08,95.0,14,14.0,42.692857,597.70,5.83,499.99,2.55
3,869.98,95.0,13,15.0,45.024286,630.34,13.31,449.99,3.64
4,356.20,95.0,3,3.0,78.556667,235.67,29.85,199.99,5.83


In [31]:
df.shape

(1020038, 32)

In [32]:
filt_df.shape

(899666, 1)

In [31]:
#Preprocessing
print('Encoding')

#Encoding
le_dict = {}
if One_Hot:
    df = df.merge(pd.get_dummies(df[encode_cols])
             ,left_index=True,right_index=True)
    for i in encode_cols:
        del df[i]
else:
    for c in encode_cols:
        encod = LabelEncoder()
        encod.fit(df[c].astype(str))
        df[c] = encod.transform(df[c].astype(str))
        le_dict[c] = dict(zip(encod.classes_, encod.transform(encod.classes_)))
    del encod

#Scaling
print('Scaling')

if Standard_Scale:
    Scalar = StandardScaler()
else:
    Scalar = MinMaxScaler()

df[num_cols] = Scalar.fit_transform(df[num_cols])

del One_Hot, LabelEncoder, Standard_Scale, Scalar
gc.collect()

Preprocessing


3612

### EDA

In [33]:
le_dict

{'Project Grade Level Category': {'Grades 3-5': 0,
  'Grades 6-8': 1,
  'Grades 9-12': 2,
  'Grades PreK-2': 3,
  'unknown': 4},
 'Project Posted Month': {'Apr': 0,
  'Aug': 1,
  'Dec': 2,
  'Feb': 3,
  'Jan': 4,
  'Jul': 5,
  'Jun': 6,
  'Mar': 7,
  'May': 8,
  'Nov': 9,
  'Oct': 10,
  'Sep': 11},
 'Project Posted Year': {'2013': 0,
  '2014': 1,
  '2015': 2,
  '2016': 3,
  '2017': 4,
  '2018': 5},
 'Project Resource Category': {'Art Supplies': 0,
  'Books': 1,
  'Classroom Basics': 2,
  'Computers & Tablets': 3,
  'Educational Kits & Games': 4,
  'Flexible Seating': 5,
  'Food, Clothing & Hygiene': 6,
  'Instructional Technology': 7,
  'Lab Equipment': 8,
  'Musical Instruments': 9,
  'Other': 10,
  'Reading Nooks, Desks & Storage': 11,
  'Sports & Exercise Equipment': 12,
  'Supplies': 13,
  'Technology': 14,
  'Trips': 15,
  'Visitors': 16},
 'Project Type': {'Professional Development': 0,
  'Student-Led': 1,
  'Teacher-Led': 2},
 'School District': {'A-C Central Cmty Unit Sd 262': 